In [45]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [46]:
# How many dishes are available and in what range of calories
DISH_COUNT = 3000
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1500

# User config - how many days, how many dishes per day, how many calories per day
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3
CALORIES_PER_DAY = 2000
# Penalty for dishes repaeating during the week
PENALTY = 400
PROBABILITY_OF_RANDOM_MUTATE = 0.1
PROBABILITY_OF_SWAP_MUTATE = 0.1
# Genetic algorithm config
POPULATION_SIZE = 200
MATING_POPULATION_SIZE = 50
ITERATIONS = 1000

#### Helper functions

In [47]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Real data

In [48]:
dishes = {}
data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

while len(dishes) != DISH_COUNT:
    random_choice = df.sample()
    if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
        dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
dishes = list(dishes.items())
print(*dishes[:DAYS_PER_MENU], sep='\n')

('Beef Top Loin Petite Roast/filet Boneless Separable Lean Only Trimmed To 1/8 Inch Fat Choice Raw', 126.65)
('Scallops Baked Or Broiled Fat Added In Cooking', 164.16)
('Cooked Pacific Herring', 360.0)
('Potato Hash Brown From School Lunch', 119.9)
('White Beans Canned Drained Low Sodium Fat Added In Cooking', 392.4)
('Oatmeal From Fast Food Fruit Flavored', 292.8)
('Lasagna With Meat', 286.34)


#### Sample random menu

In [49]:
def create_random_menu():
    menu = np.ndarray((DAYS_PER_MENU, DISHES_PER_DAY), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [50]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]

    return calories


def count_duplicates_per_day(menu):
    names = np.reshape(menu, DAYS_PER_MENU * DISHES_PER_DAY)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1)

    return attr


def score_menu(menu):
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [(attr * PENALTY) ** 2 for attr in count_duplicates_per_day(menu)]
    squares = [calories ** 2 for calories in differences]

    return round(sum(squares + penalty),2)

In [51]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 8487295.06
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                     │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1018.1  │ [('Pasta Whole Grain With Cream Sauce Restaurant', 492.5)                                                                  │
│       │            │  ('Lima Beans Dry Cooked Made With Oil', 298.8)                                                                            │
│       │            │  ('Beef Stew With Potatoes And Vegetables Including Carrots Broccoli And/or Dark-Green Leafy; Tomato-Based Sauce', 226.8)] │
├───────┼────────────┼────────────────────────────────────────────────────────────────────────

In [52]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [53]:
def create_initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        population.append(create_random_menu())
    return population

In [54]:
def genetic_algorithm():
    population = create_initial_population()
    for i in range(ITERATIONS):
        population.sort(key=score_menu)
        mating_population = population[:MATING_POPULATION_SIZE]

        offspring_population = []
        for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
            offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
            if random.random() < PROBABILITY_OF_RANDOM_MUTATE:
                offspring_population[-1] = random_mutate(offspring_population[-1])
            if random.random() < PROBABILITY_OF_SWAP_MUTATE:
                offspring_population[-1] = swap_mutate(offspring_population[-1])

        population = mating_population + offspring_population
    population.sort(key=score_menu)
    best_individual = population[0]
    return best_individual

In [55]:
best_individual = genetic_algorithm()
visualize_menu(best_individual)

Score: 10.91
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                 │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1997.16 │ [('Pineapple Candy Puerto Rican Style', 763.28)                                                                        │
│       │            │  ('Sauce Alfredo Mix Dry', 535.0)                                                                                      │
│       │            │  ('Dry Roasted Sunflower Seeds (With Salt)', 698.88)]                                                                  │
├───────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────

In [56]:
def visualize_experiment(parameters, measurements):
    headers = [parameters, "Best", "Worst"]
    table = []
    for measurement in measurements:
        table.append(measurement)
    print(tabulate(table, headers, tablefmt="fancy_grid"))

In [57]:
def experiment_population_size():
    results = []
    num_of_measurements = 5
    measurements_points = [10, 20, 50]
    for population_size in measurements_points:
        POPULATION_SIZE = population_size
        MATING_POPULATION_SIZE = population_size // 2
        best_individual_score, worst_individual_score = float('inf'), -float('inf')
        for _ in range(num_of_measurements):
            new_individual_score = score_menu(genetic_algorithm())
            best_individual_score = min(new_individual_score, best_individual_score)
            worst_individual_score = max(new_individual_score, worst_individual_score)
        results.append([population_size, best_individual_score, worst_individual_score])
    return results


visualize_experiment("Population size", experiment_population_size())

╒═══════════════════╤════════╤═════════╕
│   Population size │   Best │   Worst │
╞═══════════════════╪════════╪═════════╡
│                10 │   0.22 │   45.42 │
├───────────────────┼────────┼─────────┤
│                20 │   3.67 │  135.61 │
├───────────────────┼────────┼─────────┤
│                50 │   2.38 │   44.94 │
╘═══════════════════╧════════╧═════════╛


In [59]:
def experiment_iterations():
    results = []
    num_of_measurements = 5
    measurements_points = [10, 20, 50]
    for population_size in measurements_points:
        ITERATIONS = population_size
        best_individual_score, worst_individual_score = float('inf'), -float('inf')
        for _ in range(num_of_measurements):
            new_individual_score = score_menu(genetic_algorithm())
            best_individual_score = min(new_individual_score, best_individual_score)
            worst_individual_score = max(new_individual_score, worst_individual_score)
        results.append([population_size, best_individual_score, worst_individual_score])
    return results


visualize_experiment("Iterations", experiment_iterations())

╒══════════════╤════════╤═════════╕
│   Iterations │   Best │   Worst │
╞══════════════╪════════╪═════════╡
│           10 │   8.27 │   52.82 │
├──────────────┼────────┼─────────┤
│           20 │   1.56 │   13.49 │
├──────────────┼────────┼─────────┤
│           50 │   0.52 │   16    │
╘══════════════╧════════╧═════════╛
